In [1]:
import numpy as np

### Softmax関数
$$
y_k = \frac{\exp(s_k)}{\sum^{n}_{i=1}\exp(s_i)}
$$
softmax関数の出力は0～1の範囲となり、全て足し合わせると１になるため、出力を確率へ変換する役割がある。
学習フェーズでは次の交差エントロピー誤差関数への入力値となる。

### 交差エントロピー誤差関数
$$
L = -\sum_{k}t_k \log y_k
$$

$y_k$はsoftmax関数の出力。$t_k$は教師ラベルでone-hotベクトルを想定している

ミニバッチ処理を考慮したときの交差エントロピー誤差関数は次のように、誤差の平均を取る
$$
L = -\frac{1}{N}\sum_{n}\sum_{k}t_k \log y_k
$$

### Sigmoid関数の微分

$$
y = \frac{1}{1+\exp(-x)}\\
\frac{\partial y}{\partial x} = y(1 - y)
$$

この関係を用いて、sigmoidレイヤクラスにbackwardメソッドを実装する

In [2]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

### Affineレイヤクラスのbackward
affineレイヤでは、$y = xW+b$の計算が行われている
それぞれ、パラメータで微分する
$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y}W^{T}\\
\frac{\partial L}{\partial W} = x^{T}\frac{\partial L}{\partial y}\\
\frac{\partial L}{\partial b} = \frac{\partial L}{\partial y}
$$
$\frac{\partial L}{\partial x}$は、$x$の変化が損失$L$にどれくらい影響しているかを表している。
これらを用いて、Affineレイヤクラスにbackwardメソッドを実装する

In [3]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
    
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(x, W) + b
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0) #bはforwardでブロードキャストしているので逆伝搬の場合は合計する
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx